In [1]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the labels for emotions
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Define the CNN model
def create_cnn_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),  # Specify input shape
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(emotion_labels), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)


# Load and preprocess the dataset
def load_data():
    train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)  # For training and validation
    test_datagen = ImageDataGenerator(rescale=1./255)  # For testing

    train_data = train_datagen.flow_from_directory(
        'dataset/train',
        target_size=(48, 48),
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=32,
        subset='training'
    )
    
    val_data = train_datagen.flow_from_directory(
        'dataset/train',
        target_size=(48, 48),
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=32,
        subset='validation'
    )

    test_data = test_datagen.flow_from_directory(
        'dataset/test',
        target_size=(48, 48),
        color_mode='grayscale',
        class_mode='categorical',
        batch_size=32
    )
    
    return train_data, val_data, test_data

# Train the model
train_data, val_data, test_data = load_data()
input_shape = (48, 48, 1)
model = create_cnn_model(input_shape)
model.fit(train_data, epochs=50, validation_data=val_data)

# Save the trained model
model.save('emotion_cnn_model.keras')


Found 20099 images belonging to 7 classes.
Found 8610 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/50


c:\Users\annsu\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


629/629 ━━━━━━━━━━━━━━━━━━━━ 239s 374ms/step - accuracy: 0.2361 - loss: 1.8288 - val_accuracy: 0.3561 - val_loss: 1.6553
Epoch 2/50
629/629 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.3607 - loss: 1.6299 - val_accuracy: 0.4431 - val_loss: 1.4728
Epoch 3/50
629/629 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.4352 - loss: 1.4716 - val_accuracy: 0.4800 - val_loss: 1.3636
Epoch 4/50
629/629 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.4675 - loss: 1.3883 - val_accuracy: 0.5037 - val_loss: 1.3118
Epoch 5/50
629/629 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.4961 - loss: 1.3331 - val_accuracy: 0.4926 - val_loss: 1.3257
Epoch 6/50
629/629 ━━━━━━━━━━━━━━━━━━━━ 23s 36ms/step - accuracy: 0.5126 - loss: 1.2809 - val_accuracy: 0.5095 - val_loss: 1.2826
Epoch 7/50
629/629 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.5329 - loss: 1.2364 - val_accuracy: 0.5285 - val_loss: 1.2428
Epoch 8/50
629/629 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.5477 - loss: 1.1925 - val_accur